In [1]:
import numpy as np
import pandas as pd

In [2]:
import torch
import torch.nn as nn
from torchvision.datasets import CIFAR10
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torch.nn.functional as F

In [3]:
cuda=torch.cuda.is_available()
print(cuda)
device=torch.device('cuda' if cuda else 'cpu')

True


In [4]:
data = np.load('CWRU_dataset.npy')
labels = np.load('CWRU_lables.npy')

In [5]:
x=data[:,0:1600]

In [6]:
def mean(data,no_elements):
    X=np.zeros((data.shape[0],data.shape[1]))
    for i in range(data.shape[1]-no_elements+1):
        X[:,i]=np.mean(data[:,i:i+no_elements],axis=1)
    return X.astype(np.float16)
def median(data,no_elements):
    X=np.zeros((data.shape[0],data.shape[1]))
    for i in range(data.shape[1]-no_elements+1):
        X[:,i]=np.median(data[:,i:i+no_elements],axis=1)
    return X.astype(np.float16)
def sig_image(data,size):
    X=np.zeros((data.shape[0],size,size))
    for i in range(data.shape[0]):
        X[i]=(data[i,:].reshape(size,size))
    return X.astype(np.float16)

In [7]:
channel_mean=(mean(x,10)).astype(np.float16)
x_m=sig_image(channel_mean,40)
channel_median=(median(x,10)).astype(np.float16)
x_md=sig_image(x,40)

In [8]:
x_n=sig_image(x,40)

In [9]:
x_n.shape

(2800, 40, 40)

In [10]:
x_m.shape

(2800, 40, 40)

In [11]:
X=np.stack((x_n,x_m,x_md),axis=1).astype(np.float16)

In [12]:
X.shape

(2800, 3, 40, 40)

In [13]:
from sklearn.model_selection import train_test_split
trainx, testx, trainlabel, testlabel = train_test_split(X, labels, test_size=0.2, random_state=20)

In [14]:
sig_train, sig_test = trainx,testx
lab_train, lab_test = trainlabel,testlabel

In [15]:
sig_train = torch.from_numpy(sig_train)
sig_test = torch.from_numpy(sig_test)
lab_train= torch.from_numpy(lab_train)
lab_test = torch.from_numpy(lab_test)

In [16]:
import torch.utils.data as data_utils
batch_size = 128 
train_tensor = data_utils.TensorDataset(sig_train, lab_train) 
train_loader = data_utils.DataLoader(dataset = train_tensor, batch_size = batch_size, shuffle = True)

In [17]:
batch_size = 1024
test_tensor = data_utils.TensorDataset(sig_test, lab_test) 
test_loader = data_utils.DataLoader(dataset = test_tensor, batch_size = batch_size, shuffle = False)

In [18]:
sig_train.size()

torch.Size([2240, 3, 40, 40])

In [19]:
sig_test.size()

torch.Size([560, 3, 40, 40])

In [20]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=4,stride=1,padding = 1)
        self.mp1 = nn.MaxPool2d(kernel_size=4,stride=2)
        self.conv2 = nn.Conv2d(32,64, kernel_size=4,stride =1)
        self.mp2 = nn.MaxPool2d(kernel_size=4,stride=2)
        self.conv3 = nn.Conv2d(64,32,kernel_size = 1,stride = 1)
        self.fc1= nn.Linear(1152,256)
        self.dp1 = nn.Dropout(p=0.2)
        self.fc4 = nn.Linear(256,10)

    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp1(self.conv1(x)))
        x = F.relu(self.mp2(self.conv2(x)))
        x = F.relu((self.conv3(x)))
        x = x.view(in_size,-1)
        x = F.relu(self.fc1(x))
        x = self.dp1(x)
        x = self.fc4(x)
        
        return F.log_softmax(x, dim=1)

In [21]:
def outputSize(in_size, kernel_size, stride, padding):

    output = int((in_size - kernel_size + 2*(padding)) / stride) + 1

    return(output)

In [22]:
# def init_normal(m):
#     if type(m) == nn.Conv2d:
#         nn.init.xavier_uniform(m.weight)

In [23]:
cnn = CNN().double()
#cnn = cnn.apply(init_normal)

In [24]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn.parameters(), lr=0.001)

In [25]:
num_epochs = 100

In [27]:
total_step = len(train_loader)
loss_list = []
acc_list = []
for epoch in range(num_epochs):
    for i, (signals, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        # Run the forward pass
        signals=signals
        labels=labels
        outputs = cnn(signals.double())
        loss = criterion(outputs, labels.long())
        
        loss_list.append(loss.item())

        # Backprop and perform Adam optimisation
        
        loss.backward()
        optimizer.step()
        # Track the accuracy
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels.long()).sum().item()
        acc_list.append(correct / total)

        if (epoch+1) % 5 == 0 or epoch==0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Train Accuracy: {:.2f}%'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                          (correct / total) * 100))
        

Epoch [1/100], Step [1/18], Loss: 2.3014, Train Accuracy: 6.25%
Epoch [1/100], Step [2/18], Loss: 2.2715, Train Accuracy: 13.28%
Epoch [1/100], Step [3/18], Loss: 2.1618, Train Accuracy: 19.53%
Epoch [1/100], Step [4/18], Loss: 2.0655, Train Accuracy: 24.22%
Epoch [1/100], Step [5/18], Loss: 2.0209, Train Accuracy: 18.75%
Epoch [1/100], Step [6/18], Loss: 1.9545, Train Accuracy: 32.81%
Epoch [1/100], Step [7/18], Loss: 1.9895, Train Accuracy: 24.22%
Epoch [1/100], Step [8/18], Loss: 1.9298, Train Accuracy: 28.91%
Epoch [1/100], Step [9/18], Loss: 1.7844, Train Accuracy: 38.28%
Epoch [1/100], Step [10/18], Loss: 1.7836, Train Accuracy: 32.03%
Epoch [1/100], Step [11/18], Loss: 1.8641, Train Accuracy: 29.69%
Epoch [1/100], Step [12/18], Loss: 1.7861, Train Accuracy: 28.91%
Epoch [1/100], Step [13/18], Loss: 1.7344, Train Accuracy: 35.16%
Epoch [1/100], Step [14/18], Loss: 1.6487, Train Accuracy: 42.97%
Epoch [1/100], Step [15/18], Loss: 1.5451, Train Accuracy: 47.66%
Epoch [1/100], Step 

Epoch [30/100], Step [17/18], Loss: 0.1673, Train Accuracy: 91.41%
Epoch [30/100], Step [18/18], Loss: 0.0851, Train Accuracy: 96.88%
Epoch [35/100], Step [1/18], Loss: 0.1473, Train Accuracy: 94.53%
Epoch [35/100], Step [2/18], Loss: 0.1061, Train Accuracy: 96.09%
Epoch [35/100], Step [3/18], Loss: 0.0994, Train Accuracy: 96.09%
Epoch [35/100], Step [4/18], Loss: 0.0891, Train Accuracy: 98.44%
Epoch [35/100], Step [5/18], Loss: 0.1166, Train Accuracy: 96.09%
Epoch [35/100], Step [6/18], Loss: 0.0959, Train Accuracy: 95.31%
Epoch [35/100], Step [7/18], Loss: 0.1449, Train Accuracy: 92.97%
Epoch [35/100], Step [8/18], Loss: 0.1232, Train Accuracy: 94.53%
Epoch [35/100], Step [9/18], Loss: 0.1189, Train Accuracy: 94.53%
Epoch [35/100], Step [10/18], Loss: 0.0655, Train Accuracy: 99.22%
Epoch [35/100], Step [11/18], Loss: 0.1352, Train Accuracy: 93.75%
Epoch [35/100], Step [12/18], Loss: 0.1160, Train Accuracy: 95.31%
Epoch [35/100], Step [13/18], Loss: 0.1232, Train Accuracy: 93.75%
Epoc

Epoch [65/100], Step [15/18], Loss: 0.2917, Train Accuracy: 87.50%
Epoch [65/100], Step [16/18], Loss: 0.0235, Train Accuracy: 100.00%
Epoch [65/100], Step [17/18], Loss: 0.0360, Train Accuracy: 99.22%
Epoch [65/100], Step [18/18], Loss: 0.1063, Train Accuracy: 93.75%
Epoch [70/100], Step [1/18], Loss: 0.0191, Train Accuracy: 99.22%
Epoch [70/100], Step [2/18], Loss: 0.0235, Train Accuracy: 99.22%
Epoch [70/100], Step [3/18], Loss: 0.0215, Train Accuracy: 99.22%
Epoch [70/100], Step [4/18], Loss: 0.0112, Train Accuracy: 99.22%
Epoch [70/100], Step [5/18], Loss: 0.0093, Train Accuracy: 100.00%
Epoch [70/100], Step [6/18], Loss: 0.0161, Train Accuracy: 99.22%
Epoch [70/100], Step [7/18], Loss: 0.0079, Train Accuracy: 100.00%
Epoch [70/100], Step [8/18], Loss: 0.0244, Train Accuracy: 99.22%
Epoch [70/100], Step [9/18], Loss: 0.0199, Train Accuracy: 99.22%
Epoch [70/100], Step [10/18], Loss: 0.0184, Train Accuracy: 99.22%
Epoch [70/100], Step [11/18], Loss: 0.0276, Train Accuracy: 99.22%
E

Epoch [100/100], Step [13/18], Loss: 0.0207, Train Accuracy: 99.22%
Epoch [100/100], Step [14/18], Loss: 0.0122, Train Accuracy: 100.00%
Epoch [100/100], Step [15/18], Loss: 0.0217, Train Accuracy: 99.22%
Epoch [100/100], Step [16/18], Loss: 0.0081, Train Accuracy: 100.00%
Epoch [100/100], Step [17/18], Loss: 0.0056, Train Accuracy: 100.00%
Epoch [100/100], Step [18/18], Loss: 0.0018, Train Accuracy: 100.00%


In [28]:
total_step = len(test_loader)
print(total_step)
loss_list_test = []
acc_list_test = []
with torch.no_grad():
    for i, (signals, labels) in enumerate(test_loader):
        # Run the forward pass
        signals=signals
        labels=labels
        outputs = cnn(signals.double())
        loss = criterion(outputs, labels.long())
        loss_list_test.append(loss.item())
        if epoch%10 ==0:
            print(loss)
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels.long()).sum().item()
        acc_list_test.append(correct / total)
        if (epoch) % 1 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                          (correct / total) * 100))

1
Epoch [100/100], Step [1/1], Loss: 0.0686, Accuracy: 97.50%


In [ ]:
# if you need to save
torch.save(cnn,'cnnTC3_fold3_45.pth')